# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.4 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Session ID: 8551055c-9077-4fbc-a109-e8a2e2002b2a
Applying the following default arguments:
--glue_kernel_version 1.0.4
--enable-glue-datacatalog true
Waiting for session 8551055c-9077-4fbc-a109-e8a2e2002b2a to get into ready status...
Session 8551055c-9077-4fbc-a109-e8a2e2002b2a has been created.



# Establecemos la conexión a la api
##### Para ello le pasamos los argumentos de api host y key y establecemos conexión

In [1]:
import http.client
import ssl
import requests
import json
import pandas as pd
from pandas import json_normalize

# Creamos la conexión a la API
    

ssl_context = ssl._create_unverified_context()
conn = http.client.HTTPSConnection("v3.football.api-sports.io", context=ssl_context)

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "a7651e81f75b7516319f70fd91663320"
    }


# 1. Recopilamos los países
##### PVamos a recopilar todos los países que aparecen en la aplicación, creamos un fichero parquet que almacenaremos en una ruta de s3

In [2]:
# Función para obtener todos los códigos de país de la API
def get_data_paises()-> pd.DataFrame:

    url_paises = "https://v3.football.api-sports.io/countries"
    response = requests.request("GET", url_paises, headers=headers)
    
    data_paises = json_normalize(response.json()['response'])
    return data_paises

In [3]:
paises = get_data_paises()
paises

,name,code,flag
0,Albania,AL,https://media.api-sports.io/flags/al.svg
1,Algeria,DZ,https://media.api-sports.io/flags/dz.svg
2,Andorra,AD,https://media.api-sports.io/flags/ad.svg
3,Angola,AO,https://media.api-sports.io/flags/ao.svg
4,Antigua-And-Barbuda,AG,https://media.api-sports.io/flags/ag.svg
...,...,...,...
165,Wales,GB,https://media.api-sports.io/flags/gb.svg
166,World,None,None
167,Yemen,YE,https://media.api-sports.io/flags/ye.svg
168,Zambia,ZM,https://media.api-sports.io/flags/zm.svg


# 2.Una vez que tengamos todos los países... ...lo que vamos es crear una función para traernos todas las ligas y torneos de cada país.
##### UPDATE: Por temas de espacio sólo vamos a traernos las ligas principales (primera) de 5-6 países. ES DECIR, NO VAMOS A UTILIZAR ESTA FUNCIÓN
##### En cualquier caso, creamos la función
- Para ello el primer paso es crear la función get_data_leagues que lo que hace es recibir el código país que sobre el que sacar toda la info de todas las ligas
- Hace una llamada a la país pasándole como argumento ese pais
- Una vez recibe la respuesta, lo pasamos a json y lo "aplanamos"
- Es decir, convertimos los datos de la respuesta en formato dataframe
- Y acabamos devolviendo un dataframe

In [14]:
def get_data_leagues(id_pais)-> pd.DataFrame:

    url = "https://v3.football.api-sports.io/leagues?code=" + id_pais
    response = requests.request("GET", url, headers=headers)
    data_league = json_normalize(response.json()['response'])
    return data_league

In [15]:
# Accedemos obtenemos los datos de code de pais de la API
# Para ello hacemos una petición a la API con la url de los países a través de un bucle que va recorriendo la respuesta de la api a los países solicitados
paises = get_data_paises()
df_ligas_paises = pd.DataFrame()
for i in range(0, len(paises)):
    cod_pais = paises.loc[i, 'code']
    if cod_pais is not None:
        data_leagues_aux = get_data_leagues(cod_pais)
        df_ligas_paises = pd.concat([df_ligas_paises, data_leagues_aux], ignore_index=True)

df_ligas_paises

                                                seasons  ...                              country.flag
0     [{'year': 2011, 'start': '2011-09-25', 'end': ...  ...  https://media.api-sports.io/flags/al.svg
1     [{'year': 2011, 'start': '2011-09-25', 'end': ...  ...  https://media.api-sports.io/flags/al.svg
2     [{'year': 2011, 'start': '2011-09-10', 'end': ...  ...  https://media.api-sports.io/flags/al.svg
3     [{'year': 2011, 'start': '2011-09-10', 'end': ...  ...  https://media.api-sports.io/flags/al.svg
4     [{'year': 2020, 'start': '2020-11-01', 'end': ...  ...  https://media.api-sports.io/flags/al.svg
...                                                 ...  ...                                       ...
1136  [{'year': 2023, 'start': '2023-07-22', 'end': ...  ...  https://media.api-sports.io/flags/gb.svg
1137  [{'year': 2023, 'start': '2023-11-30', 'end': ...  ...  https://media.api-sports.io/flags/gb.svg
1138  [{'year': 2023, 'start': '2023-09-03', 'end': ...  ...  https://med

In [14]:
df_ligas_paises.to_parquet('s3://rafa-tfm-bucket/countries-apifootball/ligas_paises.parquet')

## 3. Obtenemos los datos de los equipos de cada liga

- Vamos a descargarnos los datos de todos los equipos de todas las ligas. Este valor es configurable y se puede cambiar por cualquier otro valor.
- En este caso hemos creado una función en la que se le pasa el id de la liga y el año de la temporada. 
- Para ello hemos creado una función que tomando como parámetros el id de la liga y el año de la temporada.
- Esta función accede a la país, descarga los datos y posteriormente los convierte en dataframe

## UPDATE: VAMOS A SACAR LA INFO DE SÓLO 6 LIGAS

In [16]:
id_liga_spanish = 140
id_liga_italian = 135
id_liga_german = 78
id_liga_english = 39
id_liga_french = 61
id_liga_portuguese = 94
id_liga_holland = 88
season = 2023

In [17]:
def lista_info_equipos(liga,id_season)->pd.DataFrame:

    season = int(id_season)
    id_liga = int(liga)

    url = "https://v3.football.api-sports.io/teams?league="+str(id_liga)+"&season="+str(season)

    response = requests.request("GET", url, headers=headers)
    data_info_team_league = json_normalize(response.json()['response'])
    #añadimos la liga y la temporada
    data_info_team_league['league.id'] = id_liga
    data_info_team_league['season'] = season

    return data_info_team_league

In [18]:
## Hacemos llamadas a la función con todas las ligas antes definidas

df_ligas_paises = [id_liga_spanish, id_liga_italian, id_liga_german, id_liga_english, id_liga_french, id_liga_portuguese, id_liga_holland]

df_info_equipos_ligas_paises = pd.DataFrame()
for i in range(len(df_ligas_paises)):
    id_liga = df_ligas_paises[i]
    data_equipos_aux = lista_info_equipos(id_liga,season)
    df_info_equipos_ligas_paises = pd.concat([df_info_equipos_ligas_paises, data_equipos_aux])

df_info_equipos_ligas_paises

    team.id         team.name  ... league.id season
0       529         Barcelona  ...       140   2023
1       530   Atletico Madrid  ...       140   2023
2       531     Athletic Club  ...       140   2023
3       532          Valencia  ...       140   2023
4       533        Villarreal  ...       140   2023
..      ...               ...  ...       ...    ...
13      415            Twente  ...        88   2023
14      416       FC Volendam  ...        88   2023
15      417          Waalwijk  ...        88   2023
16      419    Almere City FC  ...        88   2023
17      426  Sparta Rotterdam  ...        88   2023

[132 rows x 16 columns]


In [19]:
df_info_equipos_ligas_paises.to_parquet('s3://rafa-tfm-bucket/countries-apifootball/equipos_ligas_paises_datos.parquet')

## 4. Para cada equipo de la liga, vamos a obtener toda las estadísticas anuales acumuladas de esos equipos.

In [20]:
def get_list_stats_equipos(id_season,id_team,id_league)->pd.DataFrame:

    season = int(id_season)
    team = int(id_team)
    league = int(id_league)
    
    url_stats_equipos = "https://v3.football.api-sports.io/teams/statistics?season="+str(season)+"&team="+str(team)+"&league="+str(league)
    
    response = requests.request("GET", url_stats_equipos, headers=headers)
    equipos_stats = json_normalize(response.json()['response'])
    #Quitamos la columna lineup
    equipos_stats = equipos_stats.drop(columns=['lineups'])
    return equipos_stats

In [21]:
# Vamos a llamar a la función con todos los equipos de las ligas antes definidas. Para ello vamos a hacer un bucle que recorra todos los equipos de las ligas
df_equipos = df_info_equipos_ligas_paises[['team.id', 'league.id']]
df_equipos = df_equipos.drop_duplicates()

df_equipos_stats = pd.DataFrame()
for i in range(len(df_equipos)):
    id_team = df_equipos.iloc[i,0]
    id_league = df_equipos.iloc[i,1]
    data_equipos_stats = get_list_stats_equipos(season,id_team,id_league)
    df_equipos_stats = pd.concat([df_equipos_stats, data_equipos_stats])
    
df_equipos_stats

                             form  ...  cards.red..percentage
0   DWWWWWDWDWLWWDWLDWWWLWWDWWDWW  ...                    NaN
0   WDWLWWWWWWLWWLWLDWLWWWDLWDWLL  ...                    NaN
0   LWWDWWDLWLDWWDWDWWWWLDWDWLDWW  ...                    NaN
0    WWLLWDLLDWDWLDLLDWWWWLWLDDWL  ...                    NaN
0   LWLLWDLDLDWLLWDLLWLLDWDDDWWWW  ...                 20.00%
..                            ...  ...                    ...
0      WWWWLWWWDWDDLWWDWLDWWLWWWL  ...                    NaN
0      LLLLDLWLWLDLLLLWLDLLLDLLDL  ...                    NaN
0      LLLLWWLWLLLWDLLDLLDLWLLLWD  ...                    NaN
0      LLLLDLWWDDWDLLLWWDLWDWLDDD  ...                    NaN
0      WDWDLWLLDWLWLWLDWLDLLWLDLD  ...                    NaN

[132 rows x 124 columns]


In [22]:
df_equipos_stats.to_parquet('s3://rafa-tfm-bucket/countries-apifootball/equipos_stats.parquet')

## 5. Sacamos la lista de jugadores de cada equipo

In [ ]:
def get_list_players_team(id_team)->pd.DataFrame:

    team = int(id_team)
    
    
    url_squad_team = "https://v3.football.api-sports.io/players/squads?team="+str(team)
    
    response = requests.request("GET", url_squad_team, headers=headers)
    players_team = json_normalize(response.json()['response'][0]['players'])
    #añadimos la columna team.id
    players_team['team.id'] = team
    return players_team



In [27]:
# Hacemos una llamada a la función con todos los equipos de las ligas antes definidas. Para ello vamos a hacer un bucle que recorra todos los equipos de las ligas
# Para ello vamos a hacer un bucle que recorra todos los equipos de las ligas

df_equipos = df_info_equipos_ligas_paises[['team.id']]
df_equipos = df_equipos.drop_duplicates()


df_players_team = pd.DataFrame()
for i in range(len(df_equipos)):
    id_team = df_equipos.iloc[i,0]
    data_players_team = get_list_players_team(id_team)
    df_players_team = pd.concat([df_players_team, data_players_team])

df_players_team

        id  ... team.id
0      127  ...     529
1      126  ...     529
2   336653  ...     529
3   383647  ...     529
4   391311  ...     529
..     ...  ...     ...
28  290635  ...     426
29  359464  ...     426
30   33838  ...     426
31  278947  ...     426
32  275941  ...     426

[4192 rows x 7 columns]


In [28]:
df_players_team.to_parquet('s3://rafa-tfm-bucket/countries-apifootball/players_team.parquet')

## 6. Sacamos las estadísticas anuales acumuladas de los jugadores

In [29]:
## PENDIENTE VER COMO LO PODEMOS SACAR las STATISTICS de manera normalizada

def get_list_stats_player(id_team,id_season)->pd.DataFrame:

    team = int(id_team)
    season = int(id_season)

    #vamos a sacar todas las estadiísticas de los jugadores de un equipo en una temporada
    url_plantilla_stats = "https://v3.football.api-sports.io/players?season="+str(season)+"&team="+str(team)
    response = requests.request("GET", url_plantilla_stats, headers=headers)
    df_stats_squad_info = json_normalize(response.json()['response'])
 
    return df_stats_squad_info

In [30]:
prueba2 = get_list_stats_player(33,2023)
prueba2

                                           statistics  ...                                       player.photo
0   [{'team': {'id': 33, 'name': 'Manchester Unite...  ...  https://media.api-sports.io/football/players/3...
1   [{'team': {'id': 33, 'name': 'Manchester Unite...  ...  https://media.api-sports.io/football/players/8...
2   [{'team': {'id': 33, 'name': 'Manchester Unite...  ...  https://media.api-sports.io/football/players/8...
3   [{'team': {'id': 33, 'name': 'Manchester Unite...  ...  https://media.api-sports.io/football/players/8...
4   [{'team': {'id': 33, 'name': 'Manchester Unite...  ...  https://media.api-sports.io/football/players/9...
5   [{'team': {'id': 33, 'name': 'Manchester Unite...  ...  https://media.api-sports.io/football/players/7...
6   [{'team': {'id': 33, 'name': 'Manchester Unite...  ...  https://media.api-sports.io/football/players/1...
7   [{'team': {'id': 33, 'name': 'Manchester Unite...  ...  https://media.api-sports.io/football/players/1...
8   [{'tea

## 7. Sacamos los datos de partidos